#**Python Primer and Google Colab**



### A note on Notebooks
Notebooks are organized in a set of cells, where each cell contains a portion of code that will be run together. Some cells contain code while others can contain plain text (such as this one). 

Plain text can also be added within a code cell - this is called a 'comment' and is denoted by using a hash (#). Comments will not be run as code and they are really helpful to allow others to follow the logic of your code.

In [ ]:
# <--This '#' symbol means that this line is a 'comment'. Comments do not run or affect the script.

In [ ]:
 #print("Hello There") <- this is not processed
print("Hello World")  # <- but this is

## Data types

All data in Python has an associated 'type'.

### String
Text is passed as a string (str) in python, always inside single ('...') or double quotes ("...")

### Numbers
- Integers (int): whole numbers (positive or negative)
- Floating Point Numbers (float): real numbers with decimal positions

### Booleans
A boolean (bool) value is either a True or False

### Lists
A list is an ordered collection of items which can include different data types inside. Each item in a list is recognized by its position. Keep in mind that lists start at position 0 in python, not 1. 
A list is defined by square brackets [1,2,2,3]

In [ ]:
# Strings
type("This is a string")

In [ ]:
type("42") # <- this is also a string

In [ ]:
# Numbers
type(42)

In [ ]:
type(42.0)

In [ ]:
# Boolean
type(False)

In [ ]:
# List
type([2, 4, 6, 8])

In [ ]:
my_list = [2, 4, 6, 8]
print("List at first position (element 0):", my_list[0])

## Variables
A variable stores data values and its name must be UNIQUE.

Variables are assigned by the '=' operator and can be reassigned at any moment.

In [ ]:
my_variable = 24 * 60 * 60
print("Seconds in a day:", my_variable)

In [ ]:
my_variable = 1234
print("The new value:", my_variable)

In [ ]:
a = input("Type anything you want: ")
print("The reply was:", a)

##Conditionals

If/elif/else statements create branches in code. The block of code indented beneath them will only run if the condition is true.

In [ ]:
num = 33
answer = "I don't know yet"
print(answer)

if num > 20:
  answer = "Bigger"
else:
  answer = "Smaller"
print(answer)

##Loops

Loops allow us to repeat blocks of code. They come in two varieties: 'for' and 'while'

In [ ]:
my_list = [2, 4, 6, 8]

# For loops repeat a set number of times
for value in my_list:
  print("Value:", value, "Squared:", value*value)

In [ ]:
# While loops repeat as long as the condition is true.
# Watch out for INFINITE LOOPS, if the condition never turns false the loop won't stop

x = 1
while (x<10):
    print("run number " + str(x))
    x = x + 1

## Functions
Functions allow to call a block of code without having to rewrite it again.

Use def() to set up a new function.

In [ ]:
def name_funct(name):
    print("My name is " + name)

name_funct("Jason")

In [ ]:
name_funct("Peter")

A number of functions are already built into Python.

In [ ]:
my_list = [3, 5, 2, 1, 4]

print("List length:", len(my_list))
print("Sorted list:", sorted(my_list))

We can also use functions and variables written by others by importing them into the current workspace.

In [ ]:
from math import pi

print(round(pi, 2))

In [ ]:
import random

print(random.randint(1, 10))

Note: After a Google Colab runtime is closed, all variables will lose their values and you'll need to re-run the relevant cells.



#Connecting to Google Drive

Let's connect this Colab notebook to our Google Drive so that we can save files when we need to.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Google Colab runs in a linux environment on the cloud. 

To run an instruction as a linux command instead of a Python statement, we start the line with a '!'.

Let's make a new folder in our Google Drive.

In [ ]:
!mkdir /content/drive/MyDrive/DataScience_Workshop

Now we'll copy some additional course materials to this folder.

In [ ]:
!git clone https://github.com/ersilia-os/event-fund-ai-drug-discovery.git /content/drive/MyDrive/DataScience_Workshop

Lastly, let's check that we can read the test file in our new folder.

In [ ]:
with open("drive/MyDrive/DataScience_Workshop/data/day1/workshop_test_file.txt", "r") as f:
  print(f.read())

#**Intro to ChEMBL**

ChEMBL is an online database of bioactivity data. 

We can load a pre-downloaded malaria screening dataset with the popular 'pandas' library. Note: ChEMBL data files have fields separated by a semi-colon ';'.

In [ ]:
import pandas as pd

dataframe = pd.read_csv("drive/MyDrive/DataScience_Workshop/data/day1/gsk_3d7.csv", sep=";")
dataframe

Now let's get the subset that have < 100 nM activity and save it separately.

In [ ]:
actives = dataframe.loc[dataframe['Standard Value'] < 100]
actives.to_csv("drive/MyDrive/DataScience_Workshop/data/Day1/gsk_3d7_actives.csv")
actives

#**Visualising Compound Similarity**

Now that we have some data, let's visualise the distribution of compounds in our example library. First we need to install some additional Python packages.

In [ ]:
%%capture
!pip install rdkit-pypi
!pip install umap-learn
import sys
sys.path.append("drive/MyDrive/DataScience_Workshop/data/day1/")
from courseFunctions import *

Our compounds are represented by 100s or even 1000s of chemical descriptors. We can use 'dimensionality reduction' algorithms to reduce our data to two dimensions for visualisation. Two such methods are 'Principal Component Analysis' (PCA) and 'UMAP'.

In [ ]:
path = "drive/MyDrive/DataScience_Workshop/data/day1/"
file_list = ["gsk_3d7.csv", "gsk_3d7_actives.csv"]
my_plots = plots(path, file_list)

In [ ]:
my_plots.plot_pca()

In [ ]:
my_plots.plot_umap()

###Now it's your turn.

The St Jude Children's Research Hospital (USA) conducted a High Throughput Screen for novel anti-malarials and made public the dose-response for the ~1300 most active compounds. Let's get this plasmodium dataset from chembl and visualise it along with its actives. 


1.   Search in ChEMBL for 'plasmodium falciparum' and select 'Assays'.
2.   Sort by most-to-least number of compounds.
3.   Look for the 'St Jude Malaria Screening' dataset on the first page (ID: CHEMBL730080).
4.   Download the molecules for the assay and unzip the file.
5.   Rename the file to 'st_jude_3d7.csv'.
6.   Drag and drop the file into the 'DataScience_Workshop/data/day1/' folder on google drive.
7.   Save a new dataset of molecules with < 100 nM activity by running the code block below.
8. Plot the visualisations of the st jude data and 'actives' subset by running the next 3 cells.




In [ ]:
st_jude_dataframe = pd.read_csv("drive/MyDrive/DataScience_Workshop/data/day1/st_jude_3d7.csv", sep=";")
actives = st_jude_dataframe.loc[st_jude_dataframe['Standard Value'] < 100]
actives.to_csv("drive/MyDrive/DataScience_Workshop/data/day1/st_jude_3d7_actives.csv")

In [ ]:
path = "drive/MyDrive/DataScience_Workshop/data/day1/"
file_list = ["st_jude_3d7.csv", "st_jude_3d7_actives.csv"]
my_plots2 = plots(path, file_list)

In [ ]:
my_plots2.plot_pca()

In [ ]:
my_plots2.plot_umap()

#**Breakout Sessions:**

Next we'll move into a series of breakout sessions. The format will be to discuss the given topics as a group, take notes of any key points on a flipchart, and present these points during the feedback session afterwards.
________________________

#**Computational Tools Discussion**

Take ~10 minutes to get to know each other:

*   Give each person a chance to introduce themselves and their field of expertise.
*   Decide on a group name. 

Now take ~5 minutes for each of the following questions as a group, taking notes on your flipchart. You'll be asked for some feedback on these discussion points afterwards. 
*   What current computational tools and/or skills, if any, do you use in your research?
*   What challenges/limitations do you face that prevent you from making further use of data science tools?
*   How could some of these gaps be addressed?



#**Chemical Space Discussion** (30 mins)

20 mins discussion:

Discuss the following questions and take notes for the feedback session:
*   What do you understand by the term 'chemical space'?
*   How does this differ to the concept of 'drug-like molecules'?
*   Why should the data we use to train models be similar to the types of compounds we aim to obtain predictions for?
*   How might our training data requirements vary between a virtual screening campaign for novel chemical hits and a model to score analogues of a particular chemical series that is further along the drug discovery pipeline? 





10 mins activity:

If the notebook has restarted, re-connect to google drive with the cell below and re-install the plotting libraries in the 2nd cell.

In [ ]:
#Run this again if you need to re-connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install rdkit-pypi
!pip install umap-learn
import sys
sys.path.append("drive/MyDrive/DataScience_Workshop/data/day1/")
from courseFunctions import *

Now load the images in the next cell and run the following 6 cells to start them processing while you answer the next questions.

In these images, we have a number of chemical series in colour plotted against more general background training data in black.

*   Which of these series are outside the chemical space of our training data and why?

*   Does this make them more likely or less likely to be accurately predicted by a model trained on this data (the data in black)?

In [ ]:
#Include "Series 1"...
import requests
from IPython.display import Image
from IPython.display import display

img0 = Image("/content/drive/MyDrive/DataScience_Workshop/data/day1/PCA_chem_space.png") 
img1 = Image("/content/drive/MyDrive/DataScience_Workshop/data/day1/UMAP_chem_space.png") 

display(img0,img1)

The next 6 cells plot a set of inhibitors for the *P. falciparum* PI3K enzyme against both the GSK and St Jude datasets we plotted earlier.

*   Which dataset would you rather use to train a model to predict PI3K inhibition and why?



In [ ]:
path = "drive/MyDrive/DataScience_Workshop/data/day1/"
file_list = ["gsk_3d7.csv", "gsk_3d7_actives.csv", "pi3k_actives.csv"]
my_plots3 = plots(path, file_list)

In [ ]:
my_plots3.plot_pca()

In [ ]:
my_plots3.plot_umap()

In [ ]:
path = "drive/MyDrive/DataScience_Workshop/data/day1/"
file_list = ["st_jude_3d7.csv", "st_jude_3d7_actives.csv", "pi3k_actives.csv"]
my_plots4 = plots(path, file_list)

In [ ]:
my_plots4.plot_pca()

In [ ]:
my_plots4.plot_umap()

#**Data Cleaning** (20 mins)

*Acinetobacter baumannii* is a member of the ESKAPE pathogens - bacteria with a worrying increase in the prevalance of antibiotic resistance. ChEMBL has a number of compounds from different sources that have been screened for *A. baumannii* inhibition.

Open the "ChEMBL_A_Baumannii.csv" data file in your google drive to help you with ideas for the next section (you're not limited to this).

In your groups, brainstorm the types of inconsistencies you might need to address in drug discovery data in order to clean and standardise the dataset for machine learning. You'll provide feedback for these points during the feedback session.

Remember: We want our dataset to be complete, consistent, accurate and reliable. It should aim to have one common format.


###Extension exercise (for the brave):

Let's see some examples for basic cleaning of a dataset downloaded from ChEMBL.

**Clean a ChEMBL Dataset** (Extension exercise)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd

PATH = "/content/drive/MyDrive/DataScience_Workshop/data/day1"
FILENAME = "gsk_3d7.csv"

In [ ]:
my_dataframe = pd.read_csv(os.path.join(PATH, FILENAME), sep=";")
my_dataframe

Let's investigate which columns are present and keep only those that are of interest.

In [ ]:
my_dataframe.columns

In [ ]:
my_dataframe = my_dataframe[['Molecule ChEMBL ID', 'Smiles', 'Standard Value', 'Standard Units']]
my_dataframe

Now let's double check how many unique values are within each column. 

If there is more than one "standard unit", we'll need to standardise the values.

In [ ]:
my_dataframe.nunique()

We can use the rdkit package to standardise the SMILES strings to make duplicate detection easier.

In [ ]:
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

In [ ]:
mols = [Chem.MolFromSmiles(str(smi)) for smi in my_dataframe['Smiles'].tolist()]
canonical_smiles = [Chem.MolToSmiles(m) for m in mols]
my_dataframe["CAN_Smiles"] = canonical_smiles
my_dataframe

Let's remove rows that are missing data - either because they have no experimental data or because a canonical SMILES could not be generated.

In [ ]:
#First we change all strings to NaN (not a number) that can be detected in the next command
my_dataframe.replace('', np.nan, inplace=True)

#Drop all rows with any empty/Null/NaN values 
my_dataframe.dropna(inplace=True)

Let's check for multiple entries for the same compound.

Here we delete the duplicates but we could also, for example, take the average of the values.

In [ ]:
my_dataframe.dropna(subset=["CAN_Smiles"],inplace=True)
my_dataframe.drop_duplicates()
my_dataframe

Lastly, we can save our clean dataset.

In [ ]:
my_dataframe.to_csv(os.path.join(PATH, FILENAME + "_processed.csv"), index=False)